In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

![RNN](https://pic2.zhimg.com/80/v2-b0175ebd3419f9a11a3d0d8b00e28675_hd.jpg)

In [2]:
# Hyper-parameters
sequence_length = 28 # seq len / time step / image height
input_size = 28  # input size / image width
hidden_size = 128 # The number of features in the hidden state h. Size of W: (x.size(0), hidden_size)
num_layers = 2  # Stack two layers
num_classes = 10
batch_size = 100
num_epoch = 2
learning_rate = 0.01

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../data', 
                                           transform=transforms.ToTensor(),
                                           download=False,
                                           train=True)

test_dataset = torchvision.datasets.MNIST(root='../data', 
                                          transform=transforms.ToTensor(),
                                          train=False)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [3]:
# Recurrent Neural Netword
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # (batch, time_step, input)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        # pytorch docs: https://pytorch.org/docs/stable/nn.html
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out
    
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimize
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)  # Top fc layer directly output score of each classification
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch+1, num_epoch, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.4610757529735565
Epoch [1/2], Step [200/600], Loss: 0.2996859848499298
Epoch [1/2], Step [300/600], Loss: 0.031168194487690926
Epoch [1/2], Step [400/600], Loss: 0.12621796131134033
Epoch [1/2], Step [500/600], Loss: 0.022142700850963593
Epoch [1/2], Step [600/600], Loss: 0.05279359966516495
Epoch [2/2], Step [100/600], Loss: 0.1690695732831955
Epoch [2/2], Step [200/600], Loss: 0.0546625554561615
Epoch [2/2], Step [300/600], Loss: 0.11790063977241516
Epoch [2/2], Step [400/600], Loss: 0.17480537295341492
Epoch [2/2], Step [500/600], Loss: 0.10834701359272003
Epoch [2/2], Step [600/600], Loss: 0.11609756201505661


In [5]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == predicted).sum().item()
        total += labels.size(0)
        
    print('Test Accuracy of the model on the 10000 test images: {}%'.format(100 * correct / total))
    
# Save the model
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 100.0%
